In [1]:
pip install gensim==3.8.3

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
print('test')

test


In [2]:
# Importing the libiraries
import gensim


#check Gensim version
assert '3.8.2' <= gensim.__version__ <= '3.8.3', 'You must install Gensim 3.8.3 or 3.8.2 to be able to run LDA Mallet'


from gensim.models.wrappers import LdaMallet
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import LdaModel

import pandas as pd
import numpy as np


import math
import sys

import matplotlib.pyplot as plt

import pickle

from scipy.spatial.distance import jensenshannon
# try:
# 	from hungarian_algorithm import algorithm
# except:
# 	print('You need to install "hungarian_algorithm" package to run this code')
# 	exit()


# from post_processing import jaccard_sim, dice_sim, similarity_computation
# from pre_processing import loading_wiki_docs, prepare_corpus, preprocess_data, term_pairs_generator

# from diskcache_class import db

import platform 
#checking OS
if 'windows' in platform.system().lower():
	mallet_path = 'c:/mallet-2.0.8/bin/mallet'
elif 'linux' in platform.system().lower():
    import os
    os.environ['MALLET_HOME'] = 'mallet-2.0.8/'
    mallet_path = 'mallet-2.0.8/bin/mallet' # you should NOT need to change this
#     os.environ['MALLET_HOME'] = '../mallet-2.0.8/'
#     mallet_path = '../mallet-2.0.8/bin/mallet' # you should NOT need to change this 
else:
	mallet_path = 'mallet'

/home/zha219/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# !unzip mallet-2.0.8.zip

## define the stop words

In [7]:
#read all the stop words and add them to the list of extra stop words..
extra_stop_words = open('Stopword_list', 'r')
#addiong some stop words...
extra_stop_words_list = extra_stop_words.readlines()
extra_stopwords = set()
for item in extra_stop_words_list:
    extra_stopwords.add(item.strip())
extra_stopwords.add('amp');extra_stopwords.add('&amp');extra_stopwords.add('&amp;')

# print(extra_stopwords)

# pre-process the documents

In [5]:
#pre-processing 
#this part uses Amin's code 
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import similarities



import csv, sys
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


import pandas as pd
import numpy as np

import copy


try:
	csv.field_size_limit(sys.maxsize)
except:
	print('Error in setting maxSize for CSV output')
    
def preprocess_data(doc_list, id_list, extra_stopwords = {},len_th=4,lemmatized=False):
	'''
	Returns: a list of process dataset and origianl documents of those documents

	This function removes stop-wrods, lemmatized the documens, if stated, and eliminates the documnets 
	with lenhgth of 4 or less. 
	***These processes may result in lower number of documents than the original number. To make sure 
	you receive both the original docs and the processed doc in similar order we return both.

	parameter doc_list: a list of string (documents)
	parameter extra_stopwords: NLTK.stop_words are used, if you wish to add to that list, you can use yours.
	parameter len_th: documents with len_th and less will be removed.
	parameter lemmatized: If true, the terms will be lemmatized. **be aware that lemmatization of the documents
	will result in different topics and may need different evaluation, including NPMI, stability, or human assessment**

	'''

	# replace single smart quote with single straight quote, so as to catch stopword contractions
	doc_list = [re.sub("[\u2018\u2019]", "'", doc) for doc in doc_list] #replace qoute with regualar qoutations
	doc_list = [re.sub('\d+', '', doc) for doc in doc_list]
	doc_list = [re.sub('(\/.*?\.[\w:]+)', '', doc) for doc in doc_list]
	#doc_list = [re.sub('pdf|icon|jpg', '', doc) for doc in doc_list]
	#doc_list = [re.sub('(http(s)?:\/\/.)?(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)', '', doc) for doc in doc_list]
	doc_list = [re.sub(r"http\S+", '', doc) for doc in doc_list]

	# initialize regex tokenizer
	tokenizer = RegexpTokenizer(r'\w+')
	# create English stop words list
	en_stop = set(stopwords.words('english'))
	# add any extra stopwords
	if (len(extra_stopwords) > 0):
		en_stop = en_stop.union(extra_stopwords)

	#defining a lemmatizer
	lemmatizer = WordNetLemmatizer()

	# list for tokenized documents in loop
	texts = []
	original_docs = []
	# loop through document list
	c = 0 #counter on the document number
	for i in doc_list:
		# clean and tokenize document string
		raw = i.lower()
		tokens = tokenizer.tokenize(raw)
		stopped_tokens = []
		# remove stop words from tokens
		#stopped_tokens = [i for i in tokens if not i in en_stop and len(i)>1]
		if lemmatized:
		  for t in tokens:
		    if t not in en_stop and len(t)>1:
		      pos=nltk_tag_to_wordnet_tag(nltk.pos_tag([t])[0][1])
		      if pos:
		        stopped_tokens.append(lemmatizer.lemmatize(t,pos=pos))
		      else:
		        stopped_tokens.append(lemmatizer.lemmatize(t))
		  #     print(t,pos,nltk.pos_tag([t])[0][1])
		  # print(stopped_tokens)
		  #stopped_tokens = [lemmatizer.lemmatize(i,pos=nltk_tag_to_wordnet_tag(nltk.pos_tag([i])[0][1])) for i in tokens if not i in en_stop and len(i)>1]
		else:
		  stopped_tokens = [i for i in tokens if not i in en_stop and len(i)>1]


		# add tokens to list
		if len(stopped_tokens) >= len_th:
		  texts.append([stopped_tokens, i, id_list[c]]) # [pre-processed text (tokenized), original, id]
		  #original_docs.append([i,c])

		c += 1

	return texts

/home/zha219/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package stopwords to /home/zha219/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/zha219/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/zha219/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/zha219/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Load the wiki vocab and dict for coherence  and update Stop words

In [10]:
import pickle
with open('wiki_vocab_dict','rb') as read_file:
    wiki_vocab_dict = pickle.load(read_file)

with open('vocab_dict', 'rb') as read_file:
    vocab_dict = pickle.load(read_file)

In [11]:
# set(vocab_dict.token2id.keys())
extra_stopwords = extra_stopwords.union(set(vocab_dict.token2id.keys()).difference(set(wiki_vocab_dict.token2id.keys())))

In [ ]:
import pickle

with open('preprocessed2020doclist','rb') as read_file:
    doclist = pickle.load(read_file)

In [8]:
#import original documents of the preprocessed 2020 data
import csv
from tqdm import tqdm
completedocs = []
with open('preprocessed2020.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    for row in tqdm(csvreader):
        originaltext.append(row[1])

In [15]:
#import all documents
import csv
from tqdm import tqdm
import re
completedocs = []
with open('preprocessed2020.csv', 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    c = 0
    for row in tqdm(csvreader):
        completedocs.append([re.findall("\w+",row[0]), row[1], row[2]])

12070252it [03:57, 50752.31it/s]


In [16]:
#subsample 10 % from 2020 tweets
from random import sample
import random

random.seed(123456)
sample_size = int(len(completedocs)*.10)
completedocssampled = sample(completedocs,sample_size)
doclist = [item[0] for item in completedocssampled] #doclist is a list of list
originaltext = [item[1] for item in completedocssampled]#originaltext is a list of

In [10]:
del completedocs

In [18]:
#saving the sample documents:
with open('completedocssampled.obj', 'wb') as objwriter:
    pickle.dump(completedocssampled, objwriter)

In [17]:
len(doclist), len(originaltext)#, len(completedocs)
doclist[1:4], originaltext[1:4]

([['choice',
   'evolve',
   'repeat',
   'corinnesettle',
   'explains',
   'times',
   'hard',
   'covid'],
  ['journalists',
   'allowed',
   'courtroom',
   'due',
   'covid',
   'measures',
   'court',
   'ruled',
   'separate',
   'selmankeleş',
   'file',
   'case',
   'lives',
   'abroad',
   'warrant',
   'court',
   'rejected',
   'arifaslan',
   'demand',
   'int',
   'travel',
   'ban',
   'lifted',
   'adjourned',
   'trial'],
  ['cases', 'found', 'today', 'gb', 'nagar', 'coronavirus']],
 ['What choice will you make? Evolve or Repeat? @CorinneSettle explains how she, although at times hard, kept herself first  #pandemiclife #mentalhealth #covid',
  "Journalists weren't allowed in the courtroom due to #COVID measures. Court ruled to separate #SelmanKeleş's file from this case since he lives abroad &amp; still has a warrant against him. Court rejected #ArifAslan's demand to have his int'l travel ban lifted, adjourned the trial. ",
  ' #coronapositive cases found today in GB 

In [72]:
len(doclist), len(originaltext)

(1207025, 1207025)

In [75]:
# doclist[0:3]

In [20]:
def prepare_corpus(doc_clean,no_below=5,no_above=0.5):
	'''
	Reutrns: A dictionary of the final set of terms and document-term frequency matrix

	# adapted from https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

	parameter doc_clean: processed set of documents (type: string)
	parameter no_below: exclude words that only appear $no_below$ times or less in the whole corpus
	parameter no_above: any words included in more than $no_above$ percentage of documents will be excluded

	'''
	# Creating the term dictionary of our courpus, where every unique term is assigned an index.
	dictionary = corpora.Dictionary(doc_clean)

	dictionary.filter_extremes(no_below=no_below, no_above=no_above)
	# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
	doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
	# generate LDA model
	return dictionary,doc_term_matrix

In [ ]:
# vocab_dict, doc_term_matrix = prepare_corpus(doclist)
# N = int(len(doclist)/1000)
vocab_dict, doc_term_matrix = prepare_corpus(doclist)

In [ ]:
# doc_term_matrix, vocab_dict

In [ ]:
# import pickle

# with open('vocab_dict','wb') as save_file:
#     pickle.dump(vocab_dict,save_file)
        
# with open('doc_term_matrix','wb') as save_file:
#     pickle.dump(doc_term_matrix,save_file)

In [ ]:
#find the number of documents? 

In [ ]:
#1.Coherence Computing function

In [22]:
def compute_coherence_values(dictionary, corpus, texts, ref_dict=[], limit=25, start=5, step=5,threshold=0.10,runs = 1):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    purity_values: Average purity for each run
    contrast_values: Average of contrast for each run
    df: DataFrame df inlcudes all results and number of topics associated with those results
    """
    coherence_values = []
    coherence_std = []
    model_list = []
    purity_values = []
    contrast_values = []
    all_top_terms = []#this list will store all topics of all models as a list of top terms.
    # so with two models one with 10 and one with 20, we will have 30 of top terms
    top_terms_count_ls = [0] #this list keeps track of topics we add in each run and for each num_topics
    df = pd.DataFrame(columns=['num_topics','coherence','purity','contrast'])
    for num_topics in range(start, limit+1, step):
      model_t = []
      purity_t = []
      coherence_t = []
      contrast_t = []
      for r in range(runs):
          #model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
          model = LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary,optimize_interval = 25,iterations=2000, workers = 6) # only add it on server and not local 
          #model_t.append(model)

          #storing top_terms
          for tn in range(num_topics): 
          	tt = model.show_topic(tn,topn=20)
          	#saving top_terms and their counts
          	all_top_terms.append([i[0] for i in tt])
				  #saving counts of top_terms
          top_terms_count_ls.append(len(all_top_terms))

          #purity computation
#           topic_term_cond = get_conditional_probabilities(model,num_topics)
          topic_term = model.get_topics()#model.show_topics(num_topics = num_topics, num_words=200)
          #go over each topic and compute set of words that satisfy p(t|w)>=threshold
          pur = []
          cont = []
          for t in range(num_topics):
            pur.append(0.0)
            cont.append(0.0)
#             w_ind = np.argwhere(topic_term_cond[t,:]>=threshold)
#             pur.append(np.sum(topic_term[t,w_ind]))
#             cont.append(np.mean(topic_term_cond[t,w_ind]))
          
          #for n in range(num_topics):
            #pur.append(sum([float(i) for i in re.findall(r'\d*\.\d+',topic_term[n][1]) if float(i)>=threshold]))

          purity_t.append(np.mean(pur))
          contrast_t.append(np.mean(cont))
          # coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_npmi',processes=1)
          # coherence_value = coherencemodel.get_coherence()
          # coherence_t.append(coherence_value)
          df = df.append({'num_topics':num_topics,'purity':np.mean(pur),'contrast':np.mean(cont)},ignore_index=True)

        #stroing the results
#       model_list.append(model)
      purity_values.append(purity_t)
      #coherence_values.append(coherence_t)
      contrast_values.append(contrast_t)
      print('{0} number of topics has been processed'.format(num_topics))

    #computing coherence for all top terms at once
    cscore = CoherenceModel(topics=all_top_terms,dictionary=ref_dict,texts=texts,coherence='c_npmi',processes=7).get_coherence_per_topic() #processes are the number of threads in computing coherence

    for i in range(0,len(top_terms_count_ls)-1):
    	coherence_values.append(np.mean(cscore[top_terms_count_ls[i]:top_terms_count_ls[i+1]]))
    	coherence_std.append(np.std(cscore[top_terms_count_ls[i]:top_terms_count_ls[i+1]]))
    	#update dataframe
    df.coherence = coherence_values
    df['coherence_std'] = coherence_std
    return model_list, coherence_values, purity_values, contrast_values,df

In [ ]:
#2.1 function to load wikipedia corpus
# def loading_wiki_docs(filename:str):
# 	'''
# 	Returns: documents of wikipedia corpus

# 	loads a wikipedia text file and return documents with length>3

# 	parameter filename: name of the wikipedia text documents (type:str)
# 	'''
# 	wiki_docs = []

# 	with open(filename,'r',encoding="utf-8") as f:
# 	    d = f.readline()
# 	    wiki_docs.append(d)
# 	    while(d):
# 	      d = f.readline()
# 	      if len(d) > 3:
# 	        wiki_docs.append(d)
# 	return wiki_docs

In [ ]:
#2.2 load wikipedia for coherence computing
#topic coherence
# wiki_docs = loading_wiki_docs('../wiki_corpus/wiki_full')
#doing pre-processing on wiki-pedia documents
# pre_processed_wiki = preprocess_data(wiki_docs,[0]*len(wiki_docs))
# pre_processed_wiki = [i[0] for i in pre_processed_wiki]
# wiki_vocab_dict, _ = prepare_corpus(pre_processed_wiki)
# del wiki_docs

In [ ]:
# import pickle

# with open('pre_processed_wiki','wb') as save_file:
#     pickle.dump(pre_processed_wiki,save_file)
        
# with open('wiki_vocab_dict','wb') as save_file:
#     pickle.dump(wiki_vocab_dict,save_file)

In [ ]:
# impodoc_term_matrixe
with open('pre_processed_wiki','rb') as read_file:
    pre_processed_wiki = pickle.load(read_file)
        
with open('wiki_vocab_dict','rb') as read_file:
    wiki_vocab_dict = pickle.load(read_file)

with open('doc_term_matrix', 'rb') as read_file:
    doc_term_matrix = pickle.load(read_file)

with open('vocab_dict', 'rb') as read_file:
    vocab_dict = pickle.load(read_file)

In [ ]:
from random import sample
import random
random.seed(123456)
wiki_sample_size = int(len(pre_processed_wiki)*.05)

pre_processed_wiki = sample(pre_processed_wiki,wiki_sample_size)

In [ ]:
len(pre_processed_wiki)

In [ ]:
import seaborn as sns
def plotting_coherence(eval_df):
    ax =sns.pointplot(x='num_topics', y='coherence', data=eval_df)
    plt.title('Coherence score with Wiki docs as ref corpus')
    plt.show()
    ax = plt.errorbar(x='num_topics', y='coherence', yerr='coherence_std', data=eval_df[0:-1:3])
    plt.title('Coherence score with std within a single run')
    plt.show

In [ ]:
#3.run the LDA for the nuber of topics documents found above
from random import sample
import random
random.seed(123456)
lim = 40
st = 40
stp = 2
n_runs = 2
sample_size = 1000000
models, coherence, pur, cont,eval_df = compute_coherence_values(dictionary=vocab_dict, corpus=doc_term_matrix, texts=pre_processed_wiki,ref_dict=wiki_vocab_dict, limit=lim, start=st, step=stp,threshold=0.10,runs = n_runs)
#running on a VM machine
# eval_df.to_csv('coherence_ap_10to90.csv',index=False)
# plotting_coherence(eval_df)
eval_df.to_csv('coherence_{0}to{1}_st{2}.csv'.format(st,lim,stp))

In [ ]:
eval_df

In [ ]:
plotting_coherence(eval_df)

In [ ]:
#merge the ranges of coherence.

coherence_10_90 = pd.read_csv('coherence_10to20_st20.csv')
coherence_10_90 = coherence_10_90.append(pd.read_csv('coherence_30to50_st20.csv'))
coherence_10_90 = coherence_10_90.append(pd.read_csv('coherence_35to45_st10.csv'))
coherence_10_90 = coherence_10_90.append(pd.read_csv('coherence_55to55_st10.csv'))
coherence_10_90 = coherence_10_90.append(pd.read_csv('coherence_70to90_st20.csv'))
coherence_10_90 = coherence_10_90.append(pd.read_csv('coherence_36to54_st4.csv'))
coherence_10_90 = coherence_10_90.append(pd.read_csv('coherence_37to43_st2.csv'))
coherence_10_90 = coherence_10_90.append(pd.read_csv('coherence_38to38_st2.csv'))

In [ ]:
plotting_coherence(coherence_10_90)

In [ ]:
coherence_10_90

In [ ]:
# 4. Run LDA based on the topics number found in Step 3.
n_topics = 40; iterations = 2000
ldaMallet = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=n_topics, id2word=vocab_dict,iterations=iterations,random_seed = 54321,optimize_interval=50)

In [23]:
def get_doc_topics(model,docs,n_topics,doc_number,top_doc_n=10,show_top_doc=False):
  '''
  Computes and shows doc-topic distribution
  -----------------------------------------
  parameters:
  -----------------
  model: A LdaMallet gensim wrapper
  n_topics: number of topics that the model was trained with
  doc_number: number of documents in total
  top_doc_n: number of top documents to show for each topic
  show_top_doc: whether to show top documents for each topic or not

  returns:
  -----------------
  doc_topics_np: a numpy array of size (doc,n_topics) that shows the distribution of topics for each doc
  '''
  #test something
  doc_topics = model.load_document_topics() #loading doc-topic distribution
  doc_topics_np = np.zeros((doc_number,n_topics)) #initializing a numpy array to collect all topic-doc matrices

  docc = 0
  #reading one by one from doc_topics LDA output
  for D in doc_topics:
    doc_topics_np[docc,:] = np.asarray(D)[:,1]
    docc = docc + 1
  #top_doc_n = 10

  if show_top_doc:
    #printing top documents of each topics
    for i in range(doc_topics_np.shape[1]):
      top_doc = np.argsort(doc_topics_np[:,i])[-top_doc_n:]
      print('Topic ', i,' : ',model.show_topic(i))
      print('top docs: \n')
      for ind in reversed(range(len(top_doc))):
        print([doc_topics_np[top_doc[ind],i],docs[top_doc[ind]]])
        print("........----------------........")
      print("------------------------------------------------------------------")
      print('\n\n\n')

  return doc_topics_np

In [ ]:
n_topics = 40
doc_topics = get_doc_topics(ldaMallet,originaltext,n_topics,len(doc_term_matrix),top_doc_n=20,show_top_doc=True)

In [ ]:
with open('doc_topics.obj', 'wb') as objwriter:
    pickle.dump(doc_topics, objwriter)

In [24]:
def topic_top_terms(ldaMallet,n_topics,top_n_terms = 20):
  '''
  gets an LDAMallet model and returns top-terms in a list

  Returns: list of top terms (type:List)

  parameter ldaMallet: Trained Gensim Object for ldaMallet
  '''


  topic_term = []#np.asarray([['sample_string']*top_n]*n_topics)

  # loop through all the topics we have
  for i in range(n_topics):
    text_to_show = ''
    temp_ls = []
    # looping through the number of words we want to represent each topic ==> can do it with an iterator oved show_topic results as well.
    for j in range(top_n_terms): 
      temp_ls.append(ldaMallet.show_topic(i,topn=top_n_terms)[j][0])
    topic_term.append(temp_ls[:])

  return topic_term

In [ ]:
#get the top terms for all the topics:
topics_terms_30 = topic_top_terms(ldaMallet,n_topics,top_n_terms = 30)

In [ ]:
# topics_terms_30

In [ ]:
# topics_terms_30
with open('topics_terms_30', 'wb') as objwriter:
    pickle.dump(topics_terms_30, objwriter)

# Get the data from Data Base

In [1]:
import mysql.connector

sql_db = mysql.connector.connect(host='127.0.0.1',user = 'covidAnalysis',password = 'k34p63MbDDcZ9yf4',
                                 database = 'covid19framing')

InterfaceError: 2013: Lost connection to MySQL server during query

In [2]:
cursor = sql_db.cursor()

In [3]:
cursor.execute('show tables;')
results = cursor.fetchall()
results

[('article_author',),
 ('article_keyword',),
 ('article_organization',),
 ('articles',),
 ('organizations',),
 ('twitter',),
 ('twitter_hashtag',)]

In [4]:
cursor.execute('desc twitter')
#cursor.execute('desc twitter_hashtag')
#cursor.execute('desc articles')
results = cursor.fetchall()
results

[('id', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('organization_id', 'int(11)', 'YES', 'MUL', None, ''),
 ('user', 'varchar(20)', 'NO', '', None, ''),
 ('text', 'mediumtext', 'NO', '', None, ''),
 ('n_likes', 'int(10) unsigned', 'NO', '', None, ''),
 ('n_replies', 'int(10) unsigned', 'NO', '', None, ''),
 ('n_retweets', 'int(10) unsigned', 'NO', '', None, ''),
 ('date_posted', 'timestamp', 'NO', '', 'CURRENT_TIMESTAMP', ''),
 ('date_scraped', 'timestamp', 'NO', '', 'CURRENT_TIMESTAMP', ''),
 ('reply', 'bigint(20)', 'YES', '', None, ''),
 ('quoting', 'bigint(20)', 'YES', '', None, ''),
 ('conversation', 'bigint(20)', 'YES', '', None, ''),
 ('tweet_id', 'bigint(20)', 'NO', '', None, ''),
 ('text_preproc', 'mediumtext', 'YES', '', None, '')]

# Get the original and the pre-process documents and their ids from the DB

In [99]:
cursor.execute("select * from articles")
#cursor.execute("select * from twitter")
results = cursor.fetchall()

In [143]:
# len(results), results[0][2]

In [142]:
# len(results), results[0], results[1]

In [93]:
# doclist = [item[-1].decode().split() for item in results] ==>retreiving pre-processing docs
originaltext = [item[1].decode() for item in results] #remember that for twitter data, it should be item[3]

In [95]:
originaltext[1]

'The Transportation Security Administration (TSA) mourns the loss of Houston\nGeorge Bush Intercontinental Airport (IAH) Transportation Security Officer\nDavid Redmond, who passed away Monday after contracting COVID-19.\n\nRedmond was a Vietnam veteran and joined TSA in September 2005. He is the 19th\nTSA employee to pass away after contracting COVID-19, and the second IAH\nofficer lost in the past few weeks.\n\nTSA continues to urge that all employees get vaccinated and to follow CDC\nguidance, including mask requirements. We offer our heartfelt condolences to\nRedmond’s family, friends and coworkers.\n\n###\n\n'

# NOTE: when getting typed depencies, remove the url from texts, and the punctuations? But not the stop words... the stop words are going to be removed after parsing the grammars, and when tokens are constructed

In [11]:
# vocab_dict, doc_term_matrix = prepare_corpus(doclist)

In [12]:
# doc_term_matrix

# Get the typed depency 

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/zha219/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [1]:
from malt.malt import MaltParser # source code from nltk library

/home/zha219/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
from naive.naive_malt import get_dependency_trees
# from naive import naive_stanford 

import importlib

# importlib.reload(get_dependency_trees([['xd']]))

# res1 = get_dependency_trees([['we can have two different examples']])

# res1 = get_dependency_trees(['we can have two different examples.', 'this is the second one!'], malt_parser_version='maltparser-1.7.1', model_version='engmalt.linear-1.7.mco')
doc_reln_pairs = get_dependency_trees(originaltext, malt_parser_version='maltparser-1.7.1', model_version='engmalt.linear-1.7.mco')
# res2 = get_dependency_trees(['two different examples'])
# res1

[nltk_data] Downloading package punkt to /home/zha219/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
0it [00:00, ?it/s]-----------------------------------------------------------------------------
                          MaltParser 1.7.1                             
-----------------------------------------------------------------------------
         MALT (Models and Algorithms for Language Technology) Group          
             Vaxjo University and Uppsala University                         
                             Sweden                                          
-----------------------------------------------------------------------------

Started: Sat Apr 15 22:27:47 EDT 2023
  Transition system    : Projective
  Parser configuration : Stack
  Feature model        : eng-liblinear.xml
  Classifier           : liblinear
  Data Format          : /engmalt.linear-1.7/conllx.xml
.          	      1	      4s	    364MB
.          	     10	      4s	    375MB
.     

In [14]:
# doc_reln_pairs
# originaltext[192]
tokens = list(doc_reln_pairs.values())

In [17]:
vocab_dict, doc_term_matrix = prepare_corpus(tokens)

In [19]:
# doc_term_matrix[191]

In [1]:
# doclist[0], doc_reln_pairs[0]

In [18]:
import os

os.path.isfile('maltparser-1.7.1/maltparser-1.7.1.jar')

True

In [31]:
# !pip install graphviz
# !pip install p_tqdm
# !pip install stanza
!wget http://maltparser.org/dist/maltparser-1.7.1.zip
!unzip maltparser-1.7.1.zip
# !wget http://www.maltparser.org/mco/english_parser/engmalt.linear-1.7.mco

--2023-04-05 12:53:30--  http://maltparser.org/dist/maltparser-1.7.1.zip
Resolving maltparser.org (maltparser.org)... 195.74.38.120
Connecting to maltparser.org (maltparser.org)|195.74.38.120|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://maltparser.org/dist/maltparser-1.7.1.zip [following]
--2023-04-05 12:53:30--  https://maltparser.org/dist/maltparser-1.7.1.zip
Connecting to maltparser.org (maltparser.org)|195.74.38.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4696630 (4.5M) [application/zip]
Saving to: ‘maltparser-1.7.1.zip’

maltparser-1.7.1.zi 100%[===================>]   4.48M  5.36MB/s    in 0.8s    

2023-04-05 12:53:31 (5.36 MB/s) - ‘maltparser-1.7.1.zip’ saved [4696630/4696630]

Archive:  maltparser-1.7.1.zip
   creating: maltparser-1.7.1/
   creating: maltparser-1.7.1/appdata/
   creating: maltparser-1.7.1/appdata/dataformat/
   creating: maltparser-1.7.1/appdata/features/
   creating: maltpa

# Call the prepare corpus for the doclis


In [36]:
# vocab_dict, doc_term_matrix = prepare_corpus(tokens)

In [82]:
# doc_term_matrix
# vocab_dict, doc_term_matrix

# Run the LDA model on the new tokens (i.e., run the LDA-GR model)

In [ ]:
n_topics = 15; iterations = 2000
ldaMallet = LdaMallet(mallet_path, corpus=doc_term_matrix, num_topics=n_topics, id2word=vocab_dict,iterations=iterations,random_seed = 54321,optimize_interval=50)

# doc_topics

In [ ]:
doc_topics = get_doc_topics(ldaMallet,originaltext,n_topics,len(doc_term_matrix),top_doc_n=20,show_top_doc=True)